In [63]:
import ee
import geemap
import pandas as pd
import numpy as np

In [64]:
## authenticates the session with EE
ee.Authenticate()

# creates the connection with EE. 
ee.Initialize()

In [116]:
def get_brightness(image):
    """
    Compute the brightness of an image by summing the visible bands (B2, B3, B4).
    You can also include infrared bands (e.g., B5, B6) if you want.
    """
    brightness = image.select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8']).reduce(ee.Reducer.sum())
    return brightness

In [117]:
# Define a point of interest
roi = ee.Geometry.BBox(162.1, -77.73, 163.3, -77.59)

In [118]:
def select_brightest_pixels(image, n=10):
    """
    Select the n brightest pixels from the image based on brightness.
    """
    # Compute brightness for each pixel
    brightness = get_brightness(image)

    # Sample the image to get all pixels in the region of interest (ROI)
    sampled_points = brightness.sample(region=roi, scale=30, numPixels=1000)

    # Sort the points by brightness in descending order
    sorted_points = sampled_points.sort('sum', False)

    # Take the top n brightest points
    top_n_brightest_points = sorted_points.limit(n)

    return top_n_brightest_points

In [129]:
def get_average_of_brightest(image, n=10):
    """
    For the top n brightest pixels, calculate the mean value for each band.
    """
    brightest_points = select_brightest_pixels(image, n)
    
    # Calculate the mean for each band (e.g., B2, B3, B4, etc.)
    means = brightest_points.reduceColumns(ee.Reducer.mean(), ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8']).values()
    return ee.Dictionary(means)

In [120]:
# define soil endmember from a single image
#define ROI
roi_soil = ee.Geometry.Rectangle([163.078070, -77.625204, 163.07800, -77.625340])

#pull image
soil1 = ee.Image('LANDSAT/LC08/C02/T2_TOA/LC08_055116_20231205').select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8'])

#clip image down to ROI for soil
soil = soil1.clip(roi_soil)

#reduce down to mean for endmembers. 
soil_mean = soil.reduceRegion(ee.Reducer.mean()).values()

In [122]:
def spectral_unmixing(image, n=10):
    """
    Perform spectral unmixing using the average values of the top n brightest pixels as endmembers.
    """
    # Get the average endmember values from the brightest pixels for each image
    average_endmembers = get_average_of_brightest(image, n)

    #pull the average brightest values for each band and put under ice_mean
    ice_mean = [average_endmembers.get('B2'), average_endmembers.get('B3'), average_endmembers.get('B4'), 
                 average_endmembers.get('B5'), average_endmembers.get('B6'), average_endmembers.get('B7')]
    
    # Convert the dictionary of mean values into a list (required format for unmixing)
    endmembers = [ice_mean,soil_mean]
    
    # Perform unmixing using the average endmembers
    unmixed_image = image.unmix(endmembers, True, True)
    
    return unmixed_image.set('system:time_start', image.get('system:time_start'))

In [123]:
ids = ee.List(['LANDSAT_8_2016-11-02', 'LANDSAT_8_2016-11-04', 'LANDSAT_8_2016-11-06', 'LANDSAT_8_2016-11-08', 'LANDSAT_8_2016-11-13', 'LANDSAT_8_2016-11-15', 'LANDSAT_8_2016-12-10', 'LANDSAT_8_2016-12-13', 'LANDSAT_8_2016-12-15', 'LANDSAT_8_2016-12-17', 'LANDSAT_8_2016-12-19', 'LANDSAT_8_2016-12-24', 'LANDSAT_8_2017-01-02', 'LANDSAT_8_2017-01-11', 'LANDSAT_8_2017-01-14', 'LANDSAT_8_2017-01-18', 'LANDSAT_8_2017-01-25', 'LANDSAT_8_2017-01-27', 'LANDSAT_8_2017-01-30', 'LANDSAT_8_2017-02-01', 'LANDSAT_8_2017-11-04', 'LANDSAT_8_2017-11-07', 'LANDSAT_8_2017-11-18', 'LANDSAT_8_2017-11-20', 'LANDSAT_8_2017-11-21', 'LANDSAT_8_2017-11-25', 'LANDSAT_8_2017-11-27', 'LANDSAT_8_2017-11-30', 'LANDSAT_8_2017-12-02', 'LANDSAT_8_2017-12-07', 'LANDSAT_8_2017-12-16', 'LANDSAT_8_2017-12-23', 'LANDSAT_8_2018-01-03', 'LANDSAT_8_2018-01-05', 'LANDSAT_8_2018-01-07', 'LANDSAT_8_2018-01-10', 'LANDSAT_8_2018-01-12', 'LANDSAT_8_2018-01-14', 'LANDSAT_8_2018-01-19', 'LANDSAT_8_2018-01-26', 'LANDSAT_8_2018-01-30', 'LANDSAT_8_2018-11-05', 'LANDSAT_8_2018-11-07', 'LANDSAT_8_2018-11-08', 'LANDSAT_8_2018-11-12', 'LANDSAT_8_2018-11-17', 'LANDSAT_8_2018-11-19', 'LANDSAT_8_2018-11-23', 'LANDSAT_8_2018-11-24', 'LANDSAT_8_2018-11-26', 'LANDSAT_8_2018-11-28', 'LANDSAT_8_2018-11-30', 'LANDSAT_8_2018-12-05', 'LANDSAT_8_2018-12-30', 'LANDSAT_8_2019-01-01', 'LANDSAT_8_2019-01-04', 'LANDSAT_8_2019-01-06', 'LANDSAT_8_2019-01-10', 'LANDSAT_8_2019-01-11', 'LANDSAT_8_2019-01-15', 'LANDSAT_8_2019-01-24', 'LANDSAT_8_2019-01-26', 'LANDSAT_8_2019-11-06', 'LANDSAT_8_2019-11-08', 'LANDSAT_8_2019-11-10', 'LANDSAT_8_2019-11-11', 'LANDSAT_8_2019-11-15', 'LANDSAT_8_2019-11-17', 'LANDSAT_8_2019-11-24', 'LANDSAT_8_2019-11-26', 'LANDSAT_8_2019-11-27', 'LANDSAT_8_2019-12-03', 'LANDSAT_8_2019-12-17', 'LANDSAT_8_2019-12-24', 'LANDSAT_8_2019-12-26', 'LANDSAT_8_2019-12-31', 'LANDSAT_8_2020-01-02', 'LANDSAT_8_2020-01-11', 'LANDSAT_8_2020-01-20', 'LANDSAT_8_2020-10-30', 'LANDSAT_8_2020-11-15', 'LANDSAT_8_2020-11-17', 'LANDSAT_8_2020-11-19', 'LANDSAT_8_2020-11-24', 'LANDSAT_8_2020-11-26', 'LANDSAT_8_2020-11-28', 'LANDSAT_8_2020-11-29', 'LANDSAT_8_2020-12-01', 'LANDSAT_8_2020-12-03', 'LANDSAT_8_2020-12-08', 'LANDSAT_8_2020-12-10', 'LANDSAT_8_2020-12-14', 'LANDSAT_8_2020-12-15', 'LANDSAT_8_2020-12-21', 'LANDSAT_8_2020-12-24', 'LANDSAT_8_2020-12-26', 'LANDSAT_8_2020-12-30', 'LANDSAT_8_2021-01-06', 'LANDSAT_8_2021-01-13', 'LANDSAT_8_2021-01-15', 'LANDSAT_8_2021-01-16', 'LANDSAT_8_2021-01-18', 'LANDSAT_8_2021-01-22', 'LANDSAT_8_2021-02-01', 'LANDSAT_8_2021-10-31', 'LANDSAT_8_2021-11-04', 'LANDSAT_8_2021-11-09', 'LANDSAT_8_2021-11-11', 'LANDSAT_8_2021-11-15', 'LANDSAT_8_2021-11-20', 'LANDSAT_8_2021-12-01', 'LANDSAT_8_2021-12-02', 'LANDSAT_8_2021-12-04', 'LANDSAT_8_2021-12-06', 'LANDSAT_8_2021-12-08', 'LANDSAT_8_2021-12-11', 'LANDSAT_8_2021-12-18', 'LANDSAT_8_2021-12-20', 'LANDSAT_8_2021-12-22', 'LANDSAT_8_2021-12-24', 'LANDSAT_8_2022-01-05', 'LANDSAT_8_2022-01-07', 'LANDSAT_8_2022-01-12', 'LANDSAT_8_2022-01-14', 'LANDSAT_8_2022-01-16', 'LANDSAT_8_2022-01-19', 'LANDSAT_8_2022-01-25', 'LANDSAT_8_2022-01-28', 'LANDSAT_8_2022-11-14', 'LANDSAT_8_2022-11-16', 'LANDSAT_8_2022-11-18', 'LANDSAT_8_2022-11-19', 'LANDSAT_8_2022-11-21', 'LANDSAT_8_2022-11-23', 'LANDSAT_8_2022-11-28', 'LANDSAT_8_2022-12-04', 'LANDSAT_8_2022-12-05', 'LANDSAT_8_2022-12-11', 'LANDSAT_8_2022-12-16', 'LANDSAT_8_2022-12-18', 'LANDSAT_8_2023-01-01', 'LANDSAT_8_2023-01-03', 'LANDSAT_8_2023-01-10', 'LANDSAT_8_2023-01-15', 'LANDSAT_8_2023-01-22', 'LANDSAT_8_2023-01-24', 'LANDSAT_8_2023-11-01', 'LANDSAT_8_2023-11-17', 'LANDSAT_8_2023-11-19', 'LANDSAT_8_2023-11-21', 'LANDSAT_8_2023-12-05', 'LANDSAT_8_2023-12-07', 'LANDSAT_8_2023-12-10', 'LANDSAT_8_2023-12-19', 'LANDSAT_8_2023-12-28', 'LANDSAT_8_2023-12-30', 'LANDSAT_8_2024-01-04', 'LANDSAT_8_2024-01-11', 'LANDSAT_8_2024-01-13', 'LANDSAT_8_2024-01-22', 'LANDSAT_8_2024-01-24'])

In [124]:
#define start and end date for the filter collection command in the cell below. 
start_date = "2016-03-06"
end_date = "2025-01-01"

In [125]:
#filter step to remove multiple images per day based on the filtered array of dates above. 

def mosaic_by_date(imcol):
    # Convert the image collection to a list of images
    imlist = imcol.toList(imcol.size())
    
    # Get unique dates from the image collection
    def get_date(image):
        return ee.Image(image).date().format("YYYY-MM-dd")
    
    unique_dates = imlist.map(lambda im: get_date(im)).distinct()

    def create_mosaic(date_str):
        date = ee.Date(date_str)
        
        # Filter images for that day and create a mosaic
        mosaic = imcol.filterDate(date, date.advance(1, 'day')).mosaic()
        
        return mosaic.set({
            'system:time_start': date.millis(),
            'system:id': date.format('YYYY-MM-dd')
        })

    # Create mosaics for each unique date
    mosaic_imlist = unique_dates.map(create_mosaic)
    
    return ee.ImageCollection(mosaic_imlist)

# Example usage with an image collection (e.g., 'LANDSAT/LC08/C02/T1_L2')
s2 = ee.ImageCollection('LANDSAT/LC08/C02/T2_TOA')\
    .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8'])\
    .filterDate(start_date, end_date)\
    .map(addImageDate)\
    .filter(ee.Filter.inList("missDate", ids))\
    .filter(ee.Filter.gt('SUN_ELEVATION',20))\
    .filterBounds(roi)\
    .sort('DATE_ACQUIRED')
s3 = mosaic_by_date(s2)


In [126]:
#  clip all the images in the s3 collection down to the ROI
def clip_image(image):
    return image.clip(roi)

l8_clipped = s3.map(clip_image)

In [132]:
def process_images(image):
    # Apply spectral unmixing on the image
    unmixed_image = spectral_unmixing(image)  # Ensure this produces an image with a valid band structure
    
    #Reduce across bands (if spectrally unmixing adds multiple bands, which it shouldn't)
    reduced_image = unmixed_image.reduce(ee.Reducer.mean())

    # Instead of using getInfo(), store the date as metadata
    date = image.date().format('YYYY-MM-dd')
    
    # Store this date in the image metadata for later retrieval
    unmixed_image = reduced_image.set('acquired_date', date)
    
    # Return the unmixed image with additional metadata
    return unmixed_image

In [133]:
## apply the process images function to the filtered image collection l8_clipped
processed_images = l8_clipped.map(process_images)

print(processed_images.getInfo())

first_image = processed_images.first()
print(first_image)

EEException: Error in map(ID=4):
Collection.reduceColumns: Need 1 inputs for Reducer.mean, got 7.

In [89]:
image_viz_params = {
    'bands': ['B5', 'B4', 'B3'],
    'min': 0,
    'max': 0.5,
    'gamma': [0.95, 1.1, 1],
}

In [90]:
Map = geemap.Map(zoom=10, center=[-77.616808, 163.077952])
Map.addLayer(first_image)
#Map.addLayer(lake1)
#Map.addLayer(roi_ice)
Map

EEException: Collection.first: Error in map(ID=0):
Collection.reduceColumns: Need 1 inputs for Reducer.mean, got 7.